# <p style="background-color:gray; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 10px 100px; color:black; hight:max"> Upvote my work if you found it useful.🎯 </p>

# <p style="background-color:#CCE3F2; font-family:newtimeroman; font-size:175%; text-align:center; border-radius: 15px 50px;">CommonLit Readability Prize 📖</p>



<img src="https://image.slidesharecdn.com/80edc8ea-6538-4fab-a6b6-fb2c9714fd24-160328160353/95/contentreadability-1-638.jpg?cb=1459181240" alt="Readability" hight=50 width=800></img>

------------------------------------------
------------------------------------------


<p style="background-color:skyblue; font-family:newtimeroman; font-size:200%; text-align:center; border-radius: 10px 500px;"><b>Introduction</b></p>
The objective of this project is to build a Machine Learning model to rate the complexity of reading passages for grade 3-12 classroom use based on pattern extracted from analysing 6 descriptive features.<br>
<p style="background-color:skyblue; font-family:newtimeroman; font-size:200%; text-align:center; border-radius: 10px 500px;"><b>We have 79 descriptive features:</b></p>
<ul>
    <li>id : unique ID for excerpt.</li>
    <li>url_legal : URL of source.</li>
    <li>license : license of source material.</li>
    <li>excerpt : text to predict reading ease of.</li>
    <li>target : reading ease</li>
    <li>standard_error : measure of spread of scores among multiple raters for each excerpt.</li>  
</ul>
<p style="background-color:skyblue; font-family:newtimeroman; font-size:200%; text-align:center; border-radius: 10px 100px;"><b>Outline:</b></p>
<ul>
    <li><a href="#Phase I"><b>Phase I</b></a>
        <ul>
            <li><a href="#head-1">Data Pre-processing</a>
            <li><a href="#head-2">EDA</a>

### <b>Installing and importing packages</b>

In [ ]:
# We gonna use these packages for our analysis.

# importing
import os
import re
import pandas as pd
import matplotlib as plt
import seaborn as sns
import itertools
import nltk 
import string
from wordcloud import WordCloud
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob,Word
from collections import Counter

#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly 
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
import cufflinks as cf
import plotly.figure_factory as ff 
from plotly.offline import iplot
from plotly import tools
colors = px.colors.qualitative.Prism
pio.templates.default = "plotly_white"

In [ ]:
# see our files:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### <h3>Reading Data</h3>
Downloading and reading dataset from Kaggle. Right now, only considering 2834 rows for the exploration.

In [ ]:
#reading data
train = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
test = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.shape

In [ ]:
test.shape

<a id="head-1"></a>
<a id="head-1-3"></a>
# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">Data Preprocessing 🔧</p>
There are various tasks that need to perform in data processing like lowercasing, removing stop words, removing digits, removing URLs, HTML tags and many more.<br>
For basic cleaning and lemmatization, I am using the texthero and nltk package which offers features related to text preprocessing as well as data exploration. Code is as follows:

In [ ]:
#filtering the unwanted symbols, spaces, ....etc
to_replace_by_space = re.compile('[/(){}\[\]|@,;]')
punctuation = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
bad_symbols = re.compile('[^0-9a-z #+_]')
stopwords = set(stopwords.words('english'))

def text_prepare(text):
    '''
    text: a string
    returna modified version of the string
    '''
    text = text.lower() # lowercase text
    text = re.sub(punctuation, '',text)
    text = re.sub(to_replace_by_space, " ", text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(bad_symbols, "", text)         # delete symbols which are in BAD_SYMBOLS_RE from text
    text = " ".join([word for word in text.split(" ") if word not in stopwords]) # delete stopwords from text
    text = re.sub(' +', ' ', text)
    return text

In [ ]:
train['excerpt'] = train['excerpt'].apply(text_prepare)

<a id="head-2"></a>
# <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">Data Exploration and Analysis 🔍</p>
<p>It is important to know enough insights about your data before implementing any machine learning model or performing statistical hypothesis testing. Exploratory data analysis provide that insight about the data to you.
In this article, we will be discussing the respective python packages available and different exploratory task in the natural language processing domain one should perform.
    <b>So, let's start.....

In [ ]:
# lets define a function to plot a bar plot easily

def bar_plot(df,x,x_title,y,title,colors=None,text=None):
    fig = px.bar(x=x,
                 y=y,
                 text=text,
                 labels={x: x_title.title()},          # replaces default labels by column name
                 data_frame=df,
                 color=colors,
                 barmode='group',
                 template="simple_white",
                 color_discrete_sequence=px.colors.qualitative.Prism)
    
    texts = [df[col].values for col in y]
    for i, t in enumerate(texts):
        fig.data[i].text = t
        fig.data[i].textposition = 'inside'
        
    fig['layout'].title=title

    for trace in fig.data:
        trace.name = trace.name.replace('_',' ').title()

    fig.update_yaxes(tickprefix="", showgrid=True)

    fig.show()

In [ ]:
# lets define a function to plot a histogram plot easily

def hist_plot(df,x,title):
    fig = px.histogram(x=df[x],
                       color_discrete_sequence=colors,
                       opacity=0.8)

    fig['layout'].title=title
    fig.update_yaxes(tickprefix="", showgrid=True)
    fig.show()

> ### Target 

In [ ]:
title='Target distribution'
hist_plot(train, 'target' ,title)

> ### standard error

In [ ]:
title='Standard error distribution'
hist_plot(train, 'standard_error' ,title)

> ### License

In [ ]:
temp = train['license'].dropna()
temp = temp.value_counts().to_frame().reset_index()
title = 'Unique licenses count'
bar_plot(temp, 
         'index',
         'License',
         ['license'],
          title=title)

> ### Url legal

In [ ]:
# Using this iteratively I was able to get a full list of titles.
url_unique_list = train['url_legal'].dropna().apply(lambda x : re.findall('https?://([A-Za-z_0-9.-]+).*',x)[0]).unique()
url_unique_list

In [ ]:
url_count = {}
for sent in train['url_legal'].dropna().values:
    url = re.findall('https?://([A-Za-z_0-9.-]+).*',sent)[0]
    if url in url_count:
        url_count[url] += 1
    else:
        url_count[url] = 1
    
url_count_df = pd.DataFrame(data=url_count.items())
url_count_df = url_count_df.sort_values(by=1,ascending=False).rename(columns={0:'Site',1:'Count'})
url_count_df

In [ ]:
title = 'Unique Sites count'
bar_plot(url_count_df, 
         'Site',
         'Site',
         ['Count'],
          title=title)

## Words level analysis

In [ ]:
#Find words spreading (each word frequency)
freq_d = pd.Series(' '.join(train['excerpt']).split()).value_counts()
#Plot the words distribution
fig = px.line(freq_d,
              title='The word frequency visualization')
fig.update_layout(showlegend=False) 

In [ ]:
prepared_as_text = [line for line in train['excerpt']]
text_prepared_results = '/n'.join(prepared_as_text)

text= ' '.join(t for t in train['excerpt'])
words_list= text.split()

In [ ]:
word_freq= {}

for word in set(words_list):
    word_freq[word]= words_list.count(word)
    
#sorting the dictionary 
word_freq = dict(sorted(word_freq.items(), reverse=True, key=lambda item: item[1]))

In [ ]:
#sort the data and put it in a data frame for the visualization
word_freq_temp = dict(itertools.islice(word_freq.items(), 25))
word_freq_df = pd.DataFrame(word_freq_temp.items(),columns=['word','count']).sort_values('count',ascending=False)

In [ ]:
bar_plot(word_freq_df.reset_index(),
         'word',
         'Words',
         ['count'],
         title='Top 20 frequent words')

In [ ]:
def wordcloud(text,stopwords,ngram=1):
    # text: if ngram>1, text should be a dictionary
    wordcloud = WordCloud(width=1400, 
                          height=800,
                          random_state=2021,
                          background_color='black',
                          stopwords=stop)
    if ngram ==1:
        wordc = wordcloud.generate(' '.join(text))
    else:
        wordc = wordcloud.generate_from_frequencies(text)
    plt.figure(figsize=(20,10), facecolor='k')
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.tight_layout(pad=0)
    
wordcloud(train['excerpt'],stop)

> ### Bigrams

In [ ]:
from nltk.util import ngrams    

def get_n_grans_count(text, n_grams, min_freq):
    output = {}
    tokens = nltk.word_tokenize(text)

    #Create the n_gram
    if n_grams == 2:
        gs = nltk.bigrams(tokens)
        
    elif n_grams == 3:
        gs = nltk.trigrams(tokens)

    else:
        return 'Only 2_grams and 3_grams are supported'
    #compute frequency distribution for all the bigrams in the text
    fdist = nltk.FreqDist(gs)
    for k,v in fdist.items():
        if v > min_freq:
            index = ' '.join(k)
            output[index] = v
    
    return output

In [ ]:
two_grams = get_n_grans_count(text, n_grams=2, min_freq=10)
two_grams_df = pd.DataFrame(data=two_grams.items())
two_grams_df = two_grams_df.sort_values(by=1,ascending=False).rename(columns={0:'Two grams',1:'Count'})
two_grams_df

In [ ]:
bar_plot(two_grams_df.iloc[:20],
         'Two grams',
         'Two grams',
         ['Count'],
         title='Top 20 frequent bigram')

In [ ]:
two_grams_temp = {j.replace(' ','_'):k for j,k in two_grams.items()}

wordcloud(two_grams_temp,stop,ngram=2)

> ### Trigrams

In [ ]:
three_grams = get_n_grans_count(text, n_grams=3, min_freq=0)
three_grams_df = pd.DataFrame(data=three_grams.items())
three_grams_df = three_grams_df.sort_values(by=1,ascending=False).rename(columns={0:'Three grams',1:'Count'})
three_grams_df

In [ ]:
bar_plot(three_grams_df.iloc[:20],
         'Three grams',
         'Three grams',
         ['Count'],
         title='Top 20 frequent trigram')

In [ ]:
three_grams_temp = {j.replace(' ','_') : k for j, k in three_grams.items()}

wordcloud(three_grams_temp,stop,ngram=3)

> ### Words length

In [ ]:
words_length = {}

for word in set(words_list):
    words_length[word] = len(word)
    
words_length = dict(sorted(words_length.items(), reverse=True, key=lambda item: item[1]))
#sort the data and put it in a data frame for the visualization
word_length_temp = dict(itertools.islice(words_length.items(), 25))
words_length_df = pd.DataFrame(words_length.items(),columns=['word','count']).sort_values('count',ascending=False)

In [ ]:
hist_plot(words_length_df,
          'count',
          title='Target distribution')

> <b>So most of words have length between 4-11.<br> 
But Wait!, something is weird here. There are words with a length of more than 15. We have to verify, are these actually English words or not?

In [ ]:
longer_23 = words_length_df[words_length_df['count'] > 23]
longer_23

<h2>Sentence level analysis</h2><br>
Sentence level analysis Text statistics include sentence length distribution, minimum, maximum, and average length. To check the sentence length distribution. Code and output are as follows:

In [ ]:
train['sentence_len']= train['excerpt'].str.len()
print('Max length     : {} \nMin length     : {} \nAverage Length : {}'.\
      format(max(train['sentence_len']),min(train['sentence_len']),train['sentence_len'].mean()))

In [ ]:
#the longest sentence we have
train[train['sentence_len']==max(train['sentence_len'])]['excerpt'].values[0]

In [ ]:
#the shortest sentence we have
train[train['sentence_len']==min(train['sentence_len'])]['excerpt'].values[0]

In [ ]:
hist_plot(train,
          'sentence_len',
          title='Sentences lengh distribution with spaces')

> <b>According to distribution, most of the article lengths are in the range of 500–650 and maximum length, minimum length, and average length are 1049, 306, 617.3712.<br>The point to note here is, sentence length includes spaces between word. If you want distribution without space, you can use the below code.

In [ ]:
train['sentence_len_no_sp']= train['excerpt'].str.split().map(lambda x: len(x))

hist_plot(train,
          'sentence_len_no_sp',
          title='Sentences lengh distribution without spaces')

### <h2 align="center" style='color:red' > If you liked the notebook or learned something please <b>Upvote</b>! </h2>

<p style="background-color:skyblue; font-family:newtimeroman; font-size:200%; text-align:center; border-radius: 10px 100px;">You can also see:</p>
<ul>
<li><b><a href='https://www.kaggle.com/alaasedeeq/house-price-prediction-top-8'>✔ House Price prediction(Top 8%)</a>     
<li><b><a href='https://www.kaggle.com/alaasedeeq/predicting-the-survival-of-titanic-top-6'>Predicting the Survival of Titanic (Top 6%)</a>    
<li><b><a href='https://www.kaggle.com/alaasedeeq/prediction-of-heart-disease-machine-learning'>Prediction of Heart Disease (Machine Learning)</a>
<li><b><a href='https://www.kaggle.com/alaasedeeq/data-exploration-and-visualization-uber-data'>Data exploration and visualization(Uber Data)</a><br>
<li><b><a href='https://www.kaggle.com/alaasedeeq/hotel-booking-eda-cufflinks-and-plotly'>Hotel booking EDA (Cufflinks and plotly)
</a><br>
<li><b><a href='https://www.kaggle.com/alaasedeeq/suicide-rates-visualization-and-geographic-maps/edit/run/53135916'>Suicide Rates visualization and Geographic maps</a>
<li><b><a href='https://www.kaggle.com/alaasedeeq/superstore-data-analysis-with-plotly-clustering'>Superstore Data Analysis With Plotly(Clustering)</a>
<li><b><a href='https://www.kaggle.com/alaasedeeq/superstore-analysis-with-cufflinks-and-pandas'>Superstore Analysis With Cufflinks and pandas</a>
<li><b><a href='https://www.kaggle.com/alaasedeeq/learn-data-analysis-using-sql-and-pandas'>Learn Data Analysis using SQL and Pandas</a>
<li><b><a href='https://www.kaggle.com/alaasedeeq/european-soccer-database-with-sqlite3'>European soccer database with sqlite3</a>
<li><b><a href='https://www.kaggle.com/alaasedeeq/chinook-questions-with-sqlite'>Chinook data questions with sqlite3</a>

### <h1 align="center" style="color:red ">Thanks for reading</h1>